In [ ]:
## import cell
import os
import sys
import pandas as pd
import numpy as np
import scipy as scp
import xlrd
import xlsxwriter
import bokeh
import holoviews as hv
from bokeh.io import show, output_file
from bokeh.models import FactorRange
from bokeh.plotting import figure
import bokeh.layouts
from bokeh.layouts import column
hv.extension('bokeh', 'matplotlib')

In [ ]:
from select_df import select_df

df_foodprices = select_df(0, 'foodprices')
df_migration = select_df(0, 'migration_movements')
df_weather = select_df(0, 'temperature_and_precipitation')

In [ ]:
weather = pd.read_csv('tas_pr_1991_2015_AC.csv')
#df_pr = df_weather[['pr','year','country']] 
#df_tas = df_weather[['tas','year','country']]

def average_climate(df1, year, country):
    
    # collect desired frame
    df1 = df1.loc[df1.year == year]
    df1 = df1.loc[df1.country == country]  
    df2 = df1.mean()['tas']
    df1 = df1.mean()['pr']
    
    df_output = pd.DataFrame()
    df_output['country'] = [country]
    df_output['year'] = [year]
    df_output['average_pr'] = [df1]
    df_output['average_tas'] = [df2]
    
    return df_output

def iterate_country(df1, year1, year2):
    
    # mapping dict
    saved_dict = {'mauritania':'MRT','lesotho':'LSO','somalia':'SOM',
              'nigeria':'NGA','tanzania':'TZA','zambia':'ZMB', 
              'burundi':'BDI','afghanistan':'AFG','mali':'MLI', 
              'niger':'NER','malawi':'MWI','democratic republic congo':'ZAR',
              'cabo verde':'CPV','kenya':'KEN','gambia':'GMB',
              'sudan':'SDN','pakistan':'PAK','burkina faso':'BFA',
              'rwanda':'RWA','kenia':'KEN','senegal':'SEN',
              'cameroon':'CMR','sierra leone':'SLE','iraq':'IRQ',
              'uganda':'UGA','mozambique':'MOZ', 'congo':'COG',
              'zimbabwe':'ZWE','central african republic':'CAF',
              'ethiopia':'ETH','guinea':'GIN','liberia':'LBR',
              'djibouti':'DJI','iran  (islamic republic of)':'IRN',
              'madagascar':'MDG','lebanon':'LBN','algeria':'DZA','angola':'AGO',
              'benin':'BEN','botswana':'BWA','chad':'TCD','cote divore':'CIV','egypt':'EGY',
              'equatorial guinea':'GNQ','eritrea':'ERI','gabon':'GAB','ghana':'GHA', 
              'guinea-bissao':'GNB','israel':'ISR','jordan':'JOR','kuwait':'KWT','south africa':'ZAF', 
              'libya':'LBY','morocco':'MAR','namibia':'NAM','oman':'OMN', 
              'saudi arabia':'SAU','swasiland':'SWZ','syria':'SYR','togo':'TGO', 
              'tunis':'TUN','united arab emirates':'ARE','yemen':'YEM'
               }
    # country list
    countries = ['afghanistan','algeria','tunis','egypt','eritrea','namibia', 
                 'morocco','togo','benin','gabon','libya','syria','swasiland','south africa',
                 'botswana','yemen','oman','saudi arabia','congo',
                 'malawi','pakistan','guinea-bissao','angola',
                 'guinea','burundi','niger','somalia','pakistan','zambia','senegal', 
                 'lebanon','chad','cote divore','central african republic','liberia',
                 'cabo verde','cameroon','nigeria','zimbabwe','madagascar','lesotho',
                 'equatorial guinea','democratic republic congo','lebanon','ethiopia','kenia','ghana',
                 'israel', 'jordan','sierra leone','mali','iran  (islamic republic of)','djibouti',
                 'tanzania','sudan','rwanda','mauritania','kuwait',
                 'burkina faso','uganda','iraq','mozambique','united arab emirates']
    
    # output frame
    df_output = pd.DataFrame()
    
    # loop
    for country_key in countries:
        
        country_key = country_key.lower()
        
        if country_key in saved_dict.keys():
            
            value = saved_dict[country_key]
            
            for year in range(year1, year2+1):
                
                output = average_climate(df1,year,value)
    
                df_output = df_output.append((output),ignore_index=True)
        else:
            print("Please use valid countries, you used {}".format(country_key))

    return df_output

In [ ]:
file = iterate_country(weather,1991,2015)

file.to_csv("average_climate_data.csv")

In [ ]:
average_climate_data = pd.read_csv("average_climate_data.csv")
# averaging function
def average_climate_visualisation(file, country, year):
    
    # necessary variables
    data_inject1 = []
    data_inject2 = []
    data_inject_pr = []
    data_inject_tas = []
    yearList = []
    
    # data collection
    df1 = file.loc[file.year == year]
    df1 = file.loc[file.country == country]  
    df1_av_pr  = df1['average_pr']
    df1_av_tas = df1['average_tas']
    
    # loops over data into lists
    for pr in df1_av_pr:
        data_inject1.append(pr)
    
    for tas in df1_av_tas:
        data_inject2.append(tas)
        
    for year in df1['year']:
        yearList.append(year)

    # appending data as sets
    for i in range(len(yearList)):
        data_inject_pr.append((yearList[i], data_inject1[i]))
        data_inject_tas.append((yearList[i], data_inject2[i]))
    
    # holoviews options    
    % output size = 250
    % opts Bars.Average_precipitation (color='blue')
    % opts Curve.Average_temperature (color='red')
   
    # holoviews objects 
    bars_pr  = hv.Bars(data_inject_pr, "{}".format(country),'Average temperature and precipitation', label = 'Average_precipitation')
    bars_tas = hv.Curve(data_inject_tas, "{}".format(country),'Average temperature and precipitation', label = 'Average_temperature')

    # merge hv objects
    plot = bars_pr * bars_tas
    
    # rendere to create .html extentions
    renderer = hv.renderer('bokeh')
    renderer.save(plot, 'Average pr_tas data for {}'.format(country))
    
    return plot

def iterate_vis_average_country(file, year1, year2):
    
    # mapping dict
    saved_dict = {'mauritania':'MRT','lesotho':'LSO','somalia':'SOM',
              'nigeria':'NGA','tanzania':'TZA','zambia':'ZMB', 
              'burundi':'BDI','afghanistan':'AFG','mali':'MLI', 
              'niger':'NER','malawi':'MWI','democratic republic congo':'ZAR',
              'cabo verde':'CPV','kenya':'KEN','gambia':'GMB',
              'sudan':'SDN','pakistan':'PAK','burkina faso':'BFA',
              'rwanda':'RWA','kenia':'KEN','senegal':'SEN',
              'cameroon':'CMR','sierra leone':'SLE','iraq':'IRQ',
              'uganda':'UGA','mozambique':'MOZ', 'congo':'COG',
              'zimbabwe':'ZWE','central african republic':'CAF',
              'ethiopia':'ETH','guinea':'GIN','liberia':'LBR',
              'djibouti':'DJI','iran  (islamic republic of)':'IRN',
              'madagascar':'MDG','lebanon':'LBN','algeria':'DZA','angola':'AGO',
              'benin':'BEN','botswana':'BWA','chad':'TCD','cote divore':'CIV','egypt':'EGY',
              'equatorial guinea':'GNQ','eritrea':'ERI','gabon':'GAB','ghana':'GHA', 
              'guinea-bissao':'GNB','israel':'ISR','jordan':'JOR','kuwait':'KWT','south africa':'ZAF', 
              'libya':'LBY','morocco':'MAR','namibia':'NAM','oman':'OMN', 
              'saudi arabia':'SAU','swasiland':'SWZ','syria':'SYR','togo':'TGO', 
              'tunis':'TUN','united arab emirates':'ARE','yemen':'YEM'
               }
    # country list
    countries = ['afghanistan','algeria','tunis','egypt','eritrea','namibia', 
                 'morocco','togo','benin','gabon','libya','syria','swasiland','south africa',
                 'botswana','yemen','oman','saudi arabia', 'congo',
                 'malawi','pakistan','guinea-bissao','angola',
                 'guinea','burundi','niger','somalia','pakistan','zambia','senegal', 
                 'lebanon','chad','cote divore','central african republic','liberia',
                 'cabo verde','cameroon','nigeria','zimbabwe','madagascar','lesotho',
                 'equatorial guinea','democratic republic congo','lebanon','ethiopia','kenia','ghana',
                 'israel', 'jordan','sierra leone','mali','iran  (islamic republic of)','djibouti',
                 'tanzania','sudan','rwanda','mauritania','kuwait',
                 'burkina faso','uganda','iraq','mozambique','united arab emirates']
    
    # loop over countries
    for country_key in countries:
        
        if country_key in saved_dict.keys():
            
            country = saved_dict[country_key]
            
            for year in range(year1,year2+1):
                # change map to desired dir
                os.chdir('Average_climate_data')
                # function call
                average_climate_visualisation(file, country,year)
                # go back one dir
                os.chdir('../')


In [ ]:
iterate_vis_average_country(file, 1991, 2015)

In [ ]:
os.chdir('../')
os.getcwd()